# Learning Quantum Optics
## A Notebook for learning how to work with the package [QuantumOptics.jl](https://qojulia.org/)
### Florian Gahbauer
### February 25, 2021
### Revised March 6, 2021

In [ ]:
using LinearAlgebra
using QuantumOptics
using WignerSymbols
using Plots
#using PyPlot

## Creating Bases
In this section, we will test the SpinBasis function to create spin-1/2 bases for the electron spin (S1) and the proton spin (S2).
Note that the electron spin is sometimes referred to as S and the proton (nuclear) spin as I.
Then, we will create a composite basis to describe a Hydrogen atom in the 1s state from these two bases. 

In [ ]:
m1basis=SpinBasis(1//1)                         # basis for orbital angular momentum S1=L=|S1,m₁> with spin-1. 
m2basis=SpinBasis(1//2)                         # basis for spin S2=|S2,m₂> with spin-1/2  |↑>=[1; 0], |↓>=[0; 1] 
m1m2basis=CompositeBasis(m2basis,m1basis)       # Composite basis S1 ⊗ S2 (outer product)
#hbasis=tensor(sbasis,ibasis) # same as above, I think.
# We will call this the |L,S,mₗ,mₛ> basis or |mₗ,mₛ> for short, because we can describe it 
#by the z-component of each of the spins: |1↑>, |1↓>, |0↑>, |0↓>, |-1↑>, |-1↓> 

## Operators in the two bases
Next we will create some basic operators defined for the spin-1 and the spin-1/2 bases. 
We want the $S_z^{1 \otimes 1/2}$ operator and the identity operator in each basis. 
To make life easier, we will set $\hbar=1$.

In [ ]:
m1basisSz=0.5*sigmaz(m1basis)               # Sz operator for S1 basis
m2basisSz=0.5*sigmaz(m2basis)               # Sz operator for S2 basis: 0.5 \hbar [1 0; 0 -1]
m1basisSplus=sigmap(m1basis)                # Splus operator for S1 basis
m1basisSminus=sigmam(m1basis)               # Sminus operator for S1 basis
m2basisSplus=sigmap(m2basis)                # Splus operator for S2 basis: \hbar [0 1; 0  0]
m2basisSminus=sigmam(m2basis)               # Sminus operator for S1 basis: \hbar [0 0; 1  0]
m1basisIdentity=identityoperator(m1basis)   # Identity operator for S1 basis
m2basisIdentity=identityoperator(m2basis)   # Identity operator for S2 basis: [1 0; 0 1]
m1basisSS=0.75*m1basisIdentity              # S^2 operator for S1 basis
m2basisSS=0.75*m2basisIdentity              # S^2 operator for S2 basis: 0.75 \hbar^2 [1 0; 0 1]

## Operators in the spin(1) $\otimes$ spin(1/2) basis
Now we need to express these operators in the spin(1) $\otimes$ spin(1/2) basis. The key is the [outer product](https://en.wikipedia.org/wiki/Outer_product). 



In [ ]:
S1z=tensor(m2basisIdentity,m1basisSz)           # S1z in S1S2 basis: \hbar diagonal(0.5,0.5,-0.5,-0.5)
S2z=tensor(m2basisSz,m1basisIdentity)           # S2z in S1S2 basis: \hbar digaonal(0.5,-0.5,0.5,-0.5)
S1plus=tensor(m2basisIdentity,m1basisSplus)     # S1plus in S1S2 basis 
S1minus=tensor(m2basisIdentity,m1basisSminus)   # S1minus in S1S2 basis
S2plus=tensor(m2basisSplus,m1basisIdentity)     # S2plus in S1S2 basis 
S2minus=tensor(m2basisSminus,m1basisIdentity)   # S2minus in S1S2 basis
S1S2=S1z*S2z+0.5*(S2plus*S1minus+S2minus*S1plus)    # S1̇ S2 in S1S2 basis
#See eq. (E-19) of Chapter XII.3.b. of Cohen-Tannoudji Quantum Mechanics for S=S1, I=S2


In [ ]:
# Now define S^2 in the S1S2 basis. 
# See eq (B-18) of Chapter X of Cohen-Tannoudji Quantum Mechanics
# Should be \hbar^2 [2 0 0 0; 0 1 1 0; 0 1 1 0; 0 0 0 2]
SS=0.75*identityoperator(m1m2basis) + 0.75*identityoperator(m1m2basis) + 2*S1z*S2z +S1plus*S2minus +S1minus*S2plus 

In [ ]:
# Now define some kets in the S1S2 basis.
uu=Ket(m1m2basis,[1,0,0,0])        # | ↑, ↑ >
dd=Ket(m1m2basis,[0,0,0,1])        # | ↓, ↓ >
ud=Ket(m1m2basis,[0,1,0,0])        # | ↑, ↓ >
du=Ket(m1m2basis,[0,0,1,0])        # | ↓, ↑ >

In [ ]:
# Now define the basis vectors of the |F, m> basis, 
# where the total angular momentum F=S1+S2 and m=Fz (the z-component of F)
F1=uu                                       # | ↑, ↑ >
F2=(ud+du)/sqrt(2)                          # (| ↑, ↓ > + | ↓, ↑ >)/√2
F3=dd                                       # | ↓, ↓ >
F4=(ud-du)/sqrt(2)                          # (| ↑, ↓ > - | ↓, ↑ >)/√2
Fmbasis=SubspaceBasis([F1,F2,F3,F4])         # Now define the |F,m> basis using these basis vectors

In [ ]:
# Now define the matrix to transform a vector from the |m1,m2> basis to the |F,m> basis.
# In other words, we found |α> = C |β>, where |α> is in the |F,m> basis and β is in the |m1,m2> basis.
# A good explanation can be  found in the section on Clebsch-Gordan coefficients in
# R. Shankar, Principles of Quantum Mechanics, 2nd edition, New York, Plenum Press, 1994 (ISBN 0-306-44790-8)
# In fact, I found the entire chapter on Addition of Angular momentum (Chapter 15) very helpful.
C=[clebschgordan(1//2,1//2,1//2,1//2,1,1) clebschgordan(1//2,1//2,1//2,-1//2,1,1) clebschgordan(1//2,-1//2,1//2,1//2,1,1) clebschgordan(1//2,-1//2,1//2,-1//2,1,1);
    clebschgordan(1//2,1//2,1//2,1//2,1,0) clebschgordan(1//2,1//2,1//2,-1//2,1,0) clebschgordan(1//2,-1//2,1//2,1//2,1,0) clebschgordan(1//2,-1//2,1//2,-1//2,1,0);
    clebschgordan(1//2,1//2,1//2,1//2,1,-1) clebschgordan(1//2,1//2,1//2,-1//2,1,-1) clebschgordan(1//2,-1//2,1//2,1//2,1,-1) clebschgordan(1//2,-1//2,1//2,-1//2,1,-1);
    clebschgordan(1//2,1//2,1//2,1//2,0,0) clebschgordan(1//2,1//2,1//2,-1//2,0,0) clebschgordan(1//2,-1//2,1//2,1//2,0,0) clebschgordan(1//2,-1//2,1//2,-1//2,0,0)]

In [ ]:
# C is just a matrix for now. It is not a QuantumOptics object.
# Therefore we can just take the inverse using the appropriate 
# function from the LinearAlgebra package. 
# Following the above logic, Cinv is the transormation matrix 
# from the |F,m> basis to the |m1,m2> basis:
# |β> = Cinv |α>,
# where |β> is a ket in the |m1,m2> basis 
# and |α> is a ket in the |F,m> basis.
Cinv=inv(C)

In [ ]:
# The elements of C are type RationalRoots.
# We want them to be Float64. 
# We use the vectorized dot-function Float64 
# to convert each element of C to a Float64.
C=Float64.(C)

In [ ]:
# Now we want to convert our matrix into an operator object.
# We use the DenseOperator function from QuantumOptics.jl 
# and define a transformmatrix. 
TransformMatrix=DenseOperator(m1m2basis,C)

In [ ]:
# We also need the Hermitian conjugate of the 
# transform matrix, which we can get using the 
# dagger function from QuantumOptics.jl. 
TransformMatrixDagger=dagger(TransformMatrix)

In [ ]:
# In order to transform the operator Sz from the |m₁,m₂> basis to the
# |F,m> basis, we must multiply it on the right side by
# C† and on the left side by C. 
TransformMatrix*(S1z)*TransformMatrixDagger    # Should be Eq. E-10 in Chapter XII.E.2.a. of vol. 2 of Cohen-Tannoudji.

In [ ]:
# I think the right way to do it, actually, is to define our transform matrix
# as an operator with a left-side basis |F,m>
# and a right-side basis |m₁,m₂>. 
TransformMatrix=Operator(Fmbasis,m1m2basis,C)

In [ ]:
# Now when we take the Hermitian conjugage, the QuantumOptics package
# should handle everything for us. 
TransformMatrixDagger=dagger(TransformMatrix)

In [ ]:
Szfmbasis=TransformMatrix*(S1z)*TransformMatrixDagger    # Eq. E-10 in Chapter XII.E.2.a. of vol. 2 of Cohen-Tannoudji, Quantum Mechanics.

In [ ]:
ISfmbasis=TransformMatrix*(S1S2)*TransformMatrixDagger   # This is the operator I.S in the |F,m> basis, where it should be diagonal. 

In [ ]:
#  Now I think we are ready to calculate the Zeeman splitting. 
A=1420.405751768                # Hyperfine structure of the ground state of H, A*hbar/(2*pi), in MHz. 
q=1.6*10^(-19)                  # Electron charge in Coulomb
mₑ=9.1*10^(-31)                 # Electron mass in kg
B₀=range(0,0.03,step=0.0001)    # 1.0*10^(-4)     Magnetic field in Tesla
ω₀=-(q/(2*mₑ))*B₀/10^6          # frequency in MHz

In [ ]:
Hlist = [A*S1S2 + 2 * ω * S1z for ω=ω₀]         # The Hamiltonian is given by eq. (E-8) in Chapter XII.E.1.b. of Cohen-Tannoudji, Quantum Mechanics 
results=[eigenstates(dense(H)) for H=Hlist]; # This is a way of looping over the elements in the array Hlist to get an array called results.


In [ ]:
# Lots of information packed into the results array.
typeof(results)

In [ ]:
# Check the length. (How many entries
length(results)

In [ ]:
# Here we unpack the Eigenvalues and create 1-dimensional arrays for each Eigenvalue at different magnetic field values.
E1=[results[i][1][1] for i=range(1,length=length(results))]
E2=[results[i][1][2] for i=range(1,length=length(results))]
E3=[results[i][1][3] for i=range(1,length=length(results))]
E4=[results[i][1][4] for i=range(1,length=length(results))]
E5=[results[i][1][5] for i=range(1,length=length(results))]
E6=[results[i][1][6] for i=range(1,length=length(results))];

In [ ]:
#  Now we plot each Eigenvalue as a function of magnetic field. 
#figure(figsize=(6,3))
#xlabel(L"Magnetic Field [T]")
#ylabel(L"Energy [MHz]")
Plots.plot(B₀,[E1,E2,E3,E4,E5,E6])
#Plots.plot(B₀,E2)
#Plots.plot(B₀,E3)
#Plots.plot(B₀,E4)
#Plots.plot(B₀,E5)
#Plots.plot(B₀,E6)




In [ ]:
# This command just gives us information about the packages used in our environment.
# The information is stored in a Manifest.toml file.  
# It is useful for reproducing the environment on another machine, for example, 
# using mybinder.org.
using Pkg
Pkg.status(mode=PKGMODE_MANIFEST)

In [ ]:
# This command write a Project.toml file, which is useful if we want to recreate the environment on another machine.
pkg"status"